In [1]:
import nibabel as nib
import numpy as np
from os.path import join
import math
import matplotlib.pyplot as plt
from scipy.stats import ttest_1samp
from statsmodels.stats.multitest import multipletests
from nilearn import plotting
from nilearn.image import resample_to_img
from scipy import ndimage
from scipy.spatial.distance import cdist
from scipy.sparse import csgraph
import cvxpy as cp
from sklearn.model_selection import KFold
from itertools import product
import scipy.io as sio
import h5py

In [2]:
# def load_mat_func(path):
#     with h5py.File(path, "r") as f:
#         def _read(obj):
#             if isinstance(obj, h5py.Dataset):
#                 # print(1)
#                 data = obj[()]
#                 if isinstance(data, (bytes, np.bytes_)):
#                     return data.decode("utf-8", errors="ignore")
#                 return np.array(data)
#             # elif isinstance(obj, h5py.Group):
#             #     print(2)
#             #     return {k: _read(obj[k]) for k in obj.keys()}
#             # else:
#             #     print(3)
#             #     return obj
#         return {k: _read(f[k]) for k in f.keys()}

In [ ]:
# ses = 1
# sub = '04'
# run = 1
# mask_path = f'/mnt/TeamShare/Data_Masterfile/H20-00572_All-Dressed/PRECISIONSTIM_PD_Data_Results/fMRI_preprocessed_data/Rev_pipeline/derivatives/sub-pd0{sub}/ses-{ses}/anat/sub-pd0{sub}_ses-{ses}_T1w_brain_pve_1.nii.gz'
# white_mask = nib.load(mask_path)
# np.unique(white_mask.get_fdata())

In [ ]:
# plt.hist(white_mask.get_fdata().ravel(), bins=50)

In [ ]:
# white_mask_data = white_mask.get_fdata() > 0.9

# mask_volume = np.zeros(bold_data.shape[:3], dtype=np.uint8)
# mask_volume[white_mask_data] = 1

# # make sure it lines up with the anatomy
# mask_img = nib.Nifti1Image(mask_volume, bold_img.affine, bold_img.header)
# mask_img = resample_to_img(mask_img, anat_img, interpolation="nearest")

# # interactive overlay
# view = plotting.view_img(
#     mask_img,
#     bg_img=anat_img,
#     cmap="autumn",
#     opacity=0.7,
#     threshold=0,
#     colorbar=False
# )
# view 

In [2]:
ses = 1
sub = '04'
run = 1

base_path = '/mnt/TeamShare/Data_Masterfile/H20-00572_All-Dressed/PRECISIONSTIM_PD_Data_Results/fMRI_preprocessed_data/Rev_pipeline/derivatives'
anat_img = nib.load(f'/mnt/TeamShare/Data_Masterfile/H20-00572_All-Dressed/PRECISIONSTIM_PD_Data_Results/fMRI_preprocessed_data/Rev_pipeline/derivatives/sub-pd0{sub}/ses-{ses}/anat/sub-pd0{sub}_ses-{ses}_T1w_brain.nii.gz')

data_name = f'sub-pd0{sub}_ses-{ses}_run-{run}_task-mv_bold_corrected_smoothed_reg.nii.gz'
BOLD_path_org = join(base_path, f'sub-pd0{sub}', f'ses-{ses}', 'func', data_name)
bold_img = nib.load(BOLD_path_org)
bold_data = bold_img.get_fdata()

mask_path = f'/mnt/TeamShare/Data_Masterfile/H20-00572_All-Dressed/PRECISIONSTIM_PD_Data_Results/fMRI_preprocessed_data/Rev_pipeline/derivatives/sub-pd0{sub}/ses-{ses}/anat/sub-pd0{sub}_ses-{ses}_T1w_brain_mask.nii.gz'
back_mask = nib.load(mask_path)

mask_path = f'/mnt/TeamShare/Data_Masterfile/H20-00572_All-Dressed/PRECISIONSTIM_PD_Data_Results/fMRI_preprocessed_data/Rev_pipeline/derivatives/sub-pd0{sub}/ses-{ses}/anat/sub-pd0{sub}_ses-{ses}_T1w_brain_pve_0.nii.gz'
csf_mask = nib.load(mask_path)

mask_path = f'/mnt/TeamShare/Data_Masterfile/H20-00572_All-Dressed/PRECISIONSTIM_PD_Data_Results/fMRI_preprocessed_data/Rev_pipeline/derivatives/sub-pd0{sub}/ses-{ses}/anat/sub-pd0{sub}_ses-{ses}_T1w_brain_pve_1.nii.gz'
white_mask = nib.load(mask_path)

# print(anat_img.shape)
# print(bold_data.shape)
# print(back_mask.shape)
# print(csf_mask.shape)

In [ ]:
# ses = 1
# sub = '04'
# run = 1

# base_path = '/home/zkavian/thesis_code_git/Optim_fMRI/Optim_fMRI_new/'
# anat_img = nib.load(f'sub-pd0{sub}_ses-{ses}_T1w_brain.nii.gz')
# print(1)

# # data_name = f'fmri_sub{sub}_ses{ses}_run{run}.mat'
# # BOLD_path_org = join(base_path, data_name)
# # # bold_img = sio.loadmat(BOLD_path_org)
# # bold_mat = load_mat_func(BOLD_path_org)
# # bold_data = bold_img.get_fdata()

# base_path = '/mnt/TeamShare/Data_Masterfile/H20-00572_All-Dressed/PRECISIONSTIM_PD_Data_Results/fMRI_preprocessed_data/Rev_pipeline/derivatives'
# data_name = f'sub-pd0{sub}_ses-{ses}_run-{run}_task-mv_bold_corrected_smoothed_reg.nii.gz'
# BOLD_path_org = join(base_path, f'sub-pd0{sub}', f'ses-{ses}', 'func', data_name)
# bold_img = nib.load(BOLD_path_org)
# bold_data = bold_img.get_fdata()
# print(2)

# mask_path = f'sub-pd0{sub}_ses-{ses}_T1w_brain_mask.nii.gz'
# back_mask = nib.load(mask_path)
# print(3)

# mask_path = f'sub-pd0{sub}_ses-{ses}_T1w_brain_pve_0.nii.gz'
# csf_mask = nib.load(mask_path)
# print(4)

# # print(anat_img.shape)
# # print(bold_data.shape)
# # print(back_mask.shape)
# # print(csf_mask.shape)

In [3]:
back_mask_data = back_mask.get_fdata() > 0
csf_mask_data = csf_mask.get_fdata() > 0
white_mask_data = white_mask.get_fdata() > 0.5
mask = np.logical_and(back_mask_data, ~csf_mask_data)
mask &= ~white_mask_data
nonzero_mask = np.where(mask)
masked_bold = bold_data[nonzero_mask]

print(f"number of selected voxels after masking: {masked_bold.shape[0]/math.prod(bold_data.shape[:3])*100:.2f}%")
print('bold_data masked shape:', masked_bold.shape)

number of selected voxels after masking: 7.91%
bold_data masked shape: (619385, 850)


In [4]:
glm_dict = np.load(f'TYPED_FITHRF_GLMDENOISE_RR.npy', allow_pickle=True).item()
beta_glm = glm_dict['betasmd']
beta_run1, beta_run2 = beta_glm[:,0,0,:90], beta_glm[:,0,0,90:]
R2_run1, R2_run2 = glm_dict['R2run'][:,:,:,0], glm_dict['R2run'][:,:,:,1]

mask = np.logical_and(back_mask_data, ~csf_mask_data)
nonzero_mask = np.where(mask)
white_mask_flat = white_mask_data[nonzero_mask]
beta = beta_run1[~white_mask_flat]
R2 = R2_run1[~white_mask_flat]

lower_thr, upper_thr = np.nanpercentile(beta, [1, 99])
print(f'low_thr: {lower_thr:.2f}, high_thr: {upper_thr:.2f}') #low_thr: -4.64, high_thr: 4.60
beta_extreme_mask = np.logical_or(beta < lower_thr, beta > upper_thr)
voxels_with_extreme_beta = np.any(beta_extreme_mask, axis=1)

print(f"percentage of voxels with extreme beta values: {np.sum(voxels_with_extreme_beta)/beta.shape[0]*100:.2f}%")


low_thr: -4.41, high_thr: 4.42
percentage of voxels with extreme beta values: 7.53%


In [5]:
mask = np.logical_and(back_mask_data, ~csf_mask_data)
mask &= ~white_mask_data
nonzero_mask = np.where(mask)

In [7]:
# # extreme_beta_volume = np.full(bold_data.shape[:3], np.nan)
# # extreme_indices = np.nonzero(voxels_with_extreme_beta)[0]
# # extreme_beta_value = np.nanmean(beta[extreme_indices], axis=1)

# # extreme_voxel_coords = tuple(idx[extreme_indices] for idx in nonzero_mask)
# # extreme_beta_volume[extreme_voxel_coords] = extreme_beta_value

# plt.figure()
# plt.hist(extreme_beta_volume.ravel(), bins=100)

In [ ]:
# # # visualize beta values for voxels flagged as extreme
# # extreme_beta_volume = np.full(bold_data.shape[:3], np.nan, dtype=np.float32)

# # extreme_indices = np.nonzero(voxels_with_extreme_beta)[0]

# # extreme_beta_values = np.nanmean(beta[voxels_with_extreme_beta], axis=-1)
# # coords = tuple(axis[extreme_indices] for axis in nonzero_mask)
# # extreme_beta_volume[coords] = extreme_beta_values.astype(np.float32)

# extreme_beta_img = nib.Nifti1Image(extreme_beta_volume, bold_img.affine, bold_img.header)
# nib.save(extreme_beta_img, f'extreme_beta_sub{sub}_ses{ses}_run{run}.nii.gz')
# extreme_beta_img_rs = resample_to_img(extreme_beta_img, anat_img, interpolation='nearest')
# extreme_beta_view = plotting.view_img(
#     extreme_beta_img_rs,
#     bg_img=anat_img,
#     cmap='cold_hot',
#     symmetric_cmap=True,
#     colorbar=True,
#     title='Extreme beta voxels on anatomy'
# )
# extreme_beta_view


In [ ]:
# beta_volume = np.full(bold_data.shape[:3], np.nan, dtype=np.float32)
# beta_volume[nonzero_mask] = np.nanmean(beta, axis=-1)

# # img = nib.Nifti1Image(beta_volume, bold_img.affine, bold_img.header)
# # nib.save(img, f'sub{sub}_ses{ses}_run{run}_meanBeta.nii.gz')

In [ ]:
# from plot_func import cfs_brain_mask_plot, mean_beta_outlier_voxels, csf_mask_with_outlier, check_beta_range_and_outliers, check_avg_beta_range

# overlay_view = cfs_brain_mask_plot(nonzero_mask, mask, bold_img, anat_img)
# overlay_view
# mean_beta_outlier_voxels(beta, bold_data, bold_img, anat_img, nonzero_mask, sub, ses, run)
# overlay_img = csf_mask_with_outlier(csf_mask_data, voxels_with_extreme_beta, bold_data, bold_img, anat_img, nonzero_mask, sub, ses, run)
# nib.save(overlay_img, f'sub{sub}_ses{ses}_run{run}_outlier_csf_mask.nii.gz')
# check_beta_range_and_outliers(beta, bold_data, bold_img, anat_img, nonzero_mask)
# check_avg_beta_range(beta, bold_data, bold_img, anat_img, nonzero_mask)

In [ ]:
# after removing voxels with extreme beta values
clean_beta = beta[~voxels_with_extreme_beta]
clean_R2 = R2[~voxels_with_extreme_beta]
print('clean_beta shape:', clean_beta.shape)

# one sample t-test against 0
tvals, pvals = ttest_1samp(clean_beta, popmean=0, axis=1, nan_policy='omit')

# FDR correction
tested = np.isfinite(pvals)
alpha=0.05
rej, q, _, _ = multipletests(pvals[tested], alpha=alpha, method='fdr_bh')

n_voxel = clean_beta.shape[0]
qvals  = np.full(n_voxel, np.nan)
reject = np.zeros(n_voxel, dtype=bool)
reject[tested] = rej
qvals[tested]  = q

# reject non-active voxels
clean_active_beta = clean_beta[reject]
clean_active_R2 = clean_R2[reject]
print(f"{clean_active_beta.shape[0]/clean_beta.shape[0]*100:.2f}% of voxels are active at FDR q<{alpha}")

# # plot hist and beta values of ative voxels
# from plot_func import active_voxel_plot
# active_voxel_plot(clean_active_beta, voxels_with_extreme_beta, reject, bold_img, anat_img, nonzero_mask, sub, ses, run)

In [ ]:
plt.hist(clean_active_beta.ravel())

In [ ]:
active_indices.shape

In [ ]:
nonzero_mask[0].shape

In [21]:
# transfer back beta value on the volume
clean_mask = ~np.asarray(voxels_with_extreme_beta, dtype=bool)
clean_indices = np.nonzero(clean_mask)[0]
active_indices = clean_indices[np.asarray(reject, dtype=bool)]

spatial_shape = bold_img.shape[:3]
n_trials = clean_active_beta.shape[1]
beta_volume = np.full(spatial_shape + (n_trials,), np.nan, dtype=np.float32)

coords = tuple(axis[active_indices] for axis in nonzero_mask)
beta_volume[coords[0], coords[1], coords[2], :] = clean_active_beta.astype(np.float32)

In [23]:
def hampel_filter_image(image, window_size, threshold_factor):
    if window_size % 2 == 0:
        raise ValueError("window_size must be odd")
        
    filtered = image.astype(float).copy()
    footprint = np.ones((window_size,)*3, dtype=bool)

    print(image.shape[3])
    for t in range(image.shape[3]):
        print(t)
        vol = image[..., t]
        med = ndimage.generic_filter(vol, np.nanmedian, footprint=footprint, mode='constant', cval=np.nan)
        mad = ndimage.generic_filter(np.abs(vol - med), np.nanmedian, footprint=footprint, mode='constant', cval=np.nan)

        scaled_mad = 1.4826 * mad  # Gaussian-consistent scaling
        valid = ~np.isnan(vol)
        outliers = valid & (np.abs(vol - med) > threshold_factor * scaled_mad)
        filtered[..., t][outliers] = np.nan
    return filtered


In [ ]:
beta_volume_filter = hampel_filter_image(beta_volume, window_size=5, threshold_factor=3)

In [ ]:
# print(beta_volume_filter.shape)

# stat_img = nib.Nifti1Image(np.nanmean(beta_volume_filter, axis=-1), bold_img.affine, bold_img.header)
# stat_img = resample_to_img(stat_img, anat_img, interpolation='linear')

# view = plotting.view_img(stat_img, bg_img=anat_img, cmap='jet', colorbar=True, title='Mean beta for active voxels')
# view.save_as_html(f'mean_beta_sub{sub}_ses{ses}_run{run}_after_hampel.html')

In [ ]:
# plt.figure()
# plt.hist(beta_volume_filter.ravel(), bins=30)
# plt.show()

In [12]:
np.save("beta_volume_filter.npy",beta_volume_filter)

In [10]:
beta_volume_filter = np.load("beta_volume_filter.npy")
bold_data = bold_img.get_fdata()

In [12]:

beta_volume_filter = beta_volume_filter[45:75, 80:110, 40:70, :]
bold_data = bold_data[45:75, 80:110, 40:70, :]
bold_data.shape

(30, 30, 30, 850)

In [13]:
spatial_shape = beta_volume_filter.shape[:-1]
voxels_with_any_nan = np.zeros(spatial_shape, dtype=bool)
voxels_with_all_nan = np.ones(spatial_shape, dtype=bool)

# Sweep the time dimension once
for t in range(beta_volume_filter.shape[-1]):
    frame_nan = np.isnan(beta_volume_filter[..., t])
    voxels_with_any_nan |= frame_nan
    voxels_with_all_nan &= frame_nan

print(np.sum(voxels_with_any_nan), np.sum(voxels_with_all_nan))

n_trial = beta_volume_filter.shape[-1]
beta_volume_filter_2d = beta_volume_filter.reshape(-1, n_trial)
print(beta_volume_filter_2d.shape)
mask_2d = voxels_with_all_nan.reshape(-1)
beta_valume_clean_2d = beta_volume_filter_2d[~mask_2d]
print(beta_valume_clean_2d.shape)

20963 18437
(27000, 90)
(8563, 90)


In [ ]:
# beta_volume_filter_reshape = beta_valume_clean_2d.reshape(-1, beta_valume_clean_2d.shape[-1])
# mean_beta_filtered = np.nanmean(beta_volume_filter_reshape, axis=-1)
# print(f"Beta range: {np.nanmin(mean_beta_filtered):.2f} to {np.nanmax(mean_beta_filtered):.2f}")

# L_task = np.divide(1., np.abs(mean_beta_filtered), where=mean_beta_filtered!=0)
# np.save("L_task.npy",L_task)
# print(f"L_task range: {np.nanmin(L_task):.2f} to {np.nanmax(L_task):.2f}")

# # # plt.figure()
# # # plt.hist(L_task)
# # # plt.show()

# # # np.where((mean_beta_filtered <0.0001) & (mean_beta_filtered> 0))
# # # np.where(abs(mean_beta_filtered) >0.1)

In [ ]:
L_task.shape

In [ ]:
# finite_counts = np.sum(np.isfinite(beta_volume), axis=-1)
# tmp = finite_counts[finite_counts>80]

# plt.figure()
# plt.hist(finite_counts.ravel(), bins=30)
# plt.show()

# plt.figure()
# plt.hist(tmp)
# plt.show()

In [ ]:
bold_data.shape

In [ ]:
bold_data_selected.shape

In [ ]:
# # finite_counts = np.sum(np.isfinite(beta_volume), axis=-1)
# # min_finite_betas = 0
# # valid_mask = finite_counts > min_finite_betas
# # coords = np.argwhere(valid_mask)
# bold_data_reshape = np.reshape(bold_data, (-1, bold_data.shape[-1]))
# bold_data_selected = bold_data_reshape[~mask_2d]

# print(0)
# # bold_data_selected = np.reshape(bold_data_selected, (-1, bold_data.shape[-1]))
# num_trials= 90
# trial_len = 9
# bold_data_selected_reshape = np.zeros((bold_data_selected.shape[0], num_trials, trial_len))

# print(1)
# start = 0
# for i in range(num_trials):
#     bold_data_selected_reshape[:, i, :] = bold_data_selected[:, start:start+trial_len]
#     start += trial_len
#     if start == 270 or start == 560:
#         start += 20

# print(2)
# diff_mat = np.diff(bold_data_selected_reshape, axis=1)
# diff_mat_flat = diff_mat.reshape(diff_mat.shape[0], -1)
# L_var = np.cov(diff_mat_flat, bias=False, dtype=np.float32)
# L_var = (L_var + L_var.T) / 2 + 1e-6 * np.eye(L_var.shape[0])
# print(L_var.shape) # I need to reduce number of voxels at the end to 100,000 - 200,000 to make it computationally feasible

In [ ]:
# L_var

In [ ]:
# plt.figure()
# plt.imshow(L_var[:20,:20], cmap='hot', interpolation='nearest')
# np.save("L_var.npy",L_var)
# print("L_var range: ",  np.nanmin(L_var), np.nanmax(L_var))

In [53]:
# crop_slices = (slice(45, 75), slice(80, 110), slice(40, 70))
# crop_offset = np.array([sl.start for sl in crop_slices])
# mask_selected = (~mask_2d).reshape(bold_data.shape[:3])
# selected_linear_idx = np.flatnonzero(mask_selected)

# voxel_indices_local = np.column_stack(np.unravel_index(selected_linear_idx, bold_data.shape[:3]))
# voxel_indices = voxel_indices_local + crop_offset
# selected_world_coords = nib.affines.apply_affine(anat_img.affine, voxel_indices)
# D = cdist(selected_world_coords, selected_world_coords)
# nonzero = D[D > 0]
# sigma = np.median(nonzero) if nonzero.size else 1.0
# W = np.exp(-D**2 / (2.0 * sigma**2))
# np.fill_diagonal(W, 0.0)
# L_smooth = csgraph.laplacian(W, normed=False)


In [ ]:
# L_smooth.shape

In [ ]:
# plt.figure()
# plt.hist(L_smooth.ravel(), bins=30)

In [ ]:
# plt.figure()
# plt.imshow(L_smooth, interpolation='nearest')
# np.save("L_smooth.npy",L_smooth)
# print("L_smooth range: ",  np.nanmin(L_smooth), np.nanmax(L_smooth))

In [ ]:
beta_valume_clean_2d.shape

In [8]:
def calculate_matrices(beta_valume_clean_2d, bold_data, anat_img, mask_2d, trial_indices=None, trial_len=9):
    num_trials = beta_valume_clean_2d.shape[-1]
    if trial_indices is None:
        trial_idx = np.arange(num_trials)
    else:
        trial_idx = np.asarray(trial_indices, dtype=int).ravel()
        trial_idx = np.unique(trial_idx)

    beta_selected = beta_valume_clean_2d[:, trial_idx]
    finite_mask = np.isfinite(beta_selected)
    counts = np.count_nonzero(finite_mask, axis=-1)
    sums = np.nansum(beta_selected, axis=-1, dtype=np.float64)
    mean_beta_filtered = np.zeros(beta_selected.shape[0], dtype=beta_selected.dtype)
    valid_counts = counts > 0
    mean_beta_filtered[valid_counts] = np.asarray(sums[valid_counts] / counts[valid_counts], dtype=beta_selected.dtype)
    L_task = np.zeros_like(mean_beta_filtered)
    valid_mean = np.abs(mean_beta_filtered) > 0
    L_task[valid_mean] = 1.0 / np.abs(mean_beta_filtered[valid_mean])

###
    bold_data_reshape = np.reshape(bold_data, (-1, bold_data.shape[-1]))
    bold_data_selected = bold_data_reshape[~mask_2d]

    bold_data_selected_reshape = np.zeros((bold_data_selected.shape[0], num_trials, trial_len), dtype=bold_data_selected.dtype)
    start = 0
    for i in range(num_trials):
        end = start + trial_len
        if end > bold_data_selected.shape[1]:
            raise ValueError("BOLD data does not contain enough timepoints for all trials")
        bold_data_selected_reshape[:, i, :] = bold_data_selected[:, start:end]
        start += trial_len
        if start == 270 or start == 560:
            start += 20

    selected_BOLD_data_subset = bold_data_selected_reshape[:, trial_idx, :]

    diff_mat = np.diff(selected_BOLD_data_subset, axis=1)
    diff_mat_flat = diff_mat.reshape(diff_mat.shape[0], -1)
    finite_cols = np.all(np.isfinite(diff_mat_flat), axis=0)
    if not np.all(finite_cols):
        diff_mat_flat = diff_mat_flat[:, finite_cols]
    if diff_mat_flat.shape[1] < 2:
        raise ValueError("Not enough finite samples to compute covariance for L_var")
    L_var = np.cov(diff_mat_flat, bias=False, dtype=np.float32)
    L_var = np.nan_to_num(L_var)
    L_var = (L_var + L_var.T) / 2 + 1e-6 * np.eye(L_var.shape[0], dtype=np.float32)
####

    mask_selected = (~mask_2d).reshape(bold_data.shape[:3])
    selected_linear_idx = np.flatnonzero(mask_selected)

    voxel_indices_local = np.column_stack(np.unravel_index(selected_linear_idx, bold_data.shape[:3]))
    voxel_indices = voxel_indices_local
    selected_world_coords = nib.affines.apply_affine(anat_img.affine, voxel_indices)
    D = cdist(selected_world_coords, selected_world_coords)
    nonzero = D[D > 0]
    sigma = np.median(nonzero) if nonzero.size else 1.0
    W = np.exp(-D**2 / (2.0 * sigma**2))
    np.fill_diagonal(W, 0.0)
    L_smooth = csgraph.laplacian(W, normed=False)

    selected_BOLD_flat = selected_BOLD_data_subset.reshape(selected_BOLD_data_subset.shape[0], -1)

    return L_task, L_var, L_smooth, selected_BOLD_flat



In [8]:
# def calculate_matrices(beta_valume_clean_2d, bold_data, anat_img, mask_2d, train_idx, trial_len): 
#     # beta_valume_clean_2d shape : (num_voxels, num_trials)
#     # bold_data is 4D fMRI data
#     # mask_2d is 1D boolean array indicating which voxels to exclude (all NaN in beta values)
    
#     beta_volume_filter_reshape = beta_valume_clean_2d.reshape(-1, beta_valume_clean_2d.shape[-1])
#     mean_beta_filtered = np.nanmean(beta_volume_filter_reshape, axis=-1)
#     # print(f"Beta range: {np.nanmin(mean_beta_filtered):.2f} to {np.nanmax(mean_beta_filtered):.2f}")
#     L_task = np.divide(1., np.abs(mean_beta_filtered), where=mean_beta_filtered!=0)
#     # np.save("L_task.npy",L_task)
#     # print(f"L_task range: {np.nanmin(L_task):.2f} to {np.nanmax(L_task):.2f}")


#     bold_data_reshape = np.reshape(bold_data, (-1, bold_data.shape[-1]))
#     bold_data_selected = bold_data_reshape[~mask_2d]
#     num_trials= 90
#     # trial_len = 9
#     bold_data_selected_reshape = np.zeros((bold_data_selected.shape[0], num_trials, trial_len))
#     start = 0
#     for i in range(num_trials):
#         bold_data_selected_reshape[:, i, :] = bold_data_selected[:, start:start+trial_len]
#         start += trial_len
#         if start == 270 or start == 560:
#             start += 20
#     diff_mat = np.diff(bold_data_selected_reshape, axis=1)
#     diff_mat_flat = diff_mat.reshape(diff_mat.shape[0], -1)
#     L_var = np.cov(diff_mat_flat, bias=False, dtype=np.float32)
#     L_var = (L_var + L_var.T) / 2 + 1e-6 * np.eye(L_var.shape[0])
#     # np.save("L_var.npy",L_var)


#     crop_slices = (slice(45, 75), slice(80, 110), slice(40, 70))
#     crop_offset = np.array([sl.start for sl in crop_slices])
#     mask_selected = (~mask_2d).reshape(bold_data.shape[:3])
#     selected_linear_idx = np.flatnonzero(mask_selected)

#     voxel_indices_local = np.column_stack(np.unravel_index(selected_linear_idx, bold_data.shape[:3]))
#     voxel_indices = voxel_indices_local + crop_offset
#     selected_world_coords = nib.affines.apply_affine(anat_img.affine, voxel_indices)
#     D = cdist(selected_world_coords, selected_world_coords)
#     nonzero = D[D > 0]
#     sigma = np.median(nonzero) if nonzero.size else 1.0
#     W = np.exp(-D**2 / (2.0 * sigma**2))
#     np.fill_diagonal(W, 0.0)
#     L_smooth = csgraph.laplacian(W, normed=False)
#     # np.save("L_smooth.npy",L_smooth)   
    
#     return L_task, L_var, L_smooth

In [14]:
def calculate_matrices(beta_valume_clean_2d, bold_data, anat_img, mask_2d, trial_indices=None, trial_len=9):
    num_trials = beta_valume_clean_2d.shape[-1]
    trial_idx = np.arange(num_trials) if trial_indices is None else np.unique(np.asarray(trial_indices, int).ravel())

    # ----- L_task (same idea as yours) -----
    beta_selected = beta_valume_clean_2d[:, trial_idx]
    counts = np.count_nonzero(np.isfinite(beta_selected), axis=-1)
    sums = np.nansum(beta_selected, axis=-1, dtype=np.float64)
    mean_beta = np.zeros(beta_selected.shape[0], dtype=np.float32)
    m = counts > 0
    mean_beta[m] = (sums[m] / counts[m]).astype(np.float32)
    L_task = np.zeros_like(mean_beta, dtype=np.float32)
    v = np.abs(mean_beta) > 0
    L_task[v] = (1.0 / np.abs(mean_beta[v])).astype(np.float32)

    # ----- reshape BOLD into trials -----
    bold_data_reshape = bold_data.reshape(-1, bold_data.shape[-1])
    bold_data_selected = bold_data_reshape[~mask_2d]         # keep voxels of interest
    bold_data_selected_reshape = np.zeros((bold_data_selected.shape[0], num_trials, trial_len), dtype=np.float32)
    start = 0
    for i in range(num_trials):
        end = start + trial_len
        if end > bold_data_selected.shape[1]:
            raise ValueError("BOLD data does not contain enough timepoints for all trials")
        bold_data_selected_reshape[:, i, :] = bold_data_selected[:, start:end]
        start += trial_len
        if start == 270 or start == 560:   # your skips
            start += 20
    X = bold_data_selected_reshape[:, trial_idx, :]          # [Nvox, Ntrials, T]

    # ----- L_var: variance of trial differences, as sparse diagonal -----
    diff_mat = np.diff(X, axis=1)                            # [Nvox, Ntrials-1, T]
    diff_mat = diff_mat.reshape(diff_mat.shape[0], -1)       # [Nvox, (Ntrials-1)*T]
    diff_mat = diff_mat[:, np.all(np.isfinite(diff_mat), axis=0)]
    if diff_mat.shape[1] < 2:
        raise ValueError("Not enough finite samples to compute L_var")
    var_vec = np.nanvar(diff_mat, axis=1, ddof=1).astype(np.float32)  # per-voxel variance
    L_var = sp.diags(var_vec, format='csc')                           # sparse diagonal (PSD)

    # ----- L_smooth: sparse 6-neighbor Laplacian on the voxel grid -----
    mask3d = (~mask_2d).reshape(bold_data.shape[:3])
    idx = -np.ones(mask3d.shape, dtype=np.int64)
    idx[mask3d] = np.arange(mask3d.sum())
    rows, cols = [], []
    for ax in range(3):
        s1 = [slice(None)]*3; s2 = [slice(None)]*3
        s1[ax] = slice(1, None); s2[ax] = slice(0, -1)
        a, b = idx[tuple(s1)], idx[tuple(s2)]
        m = (a != -1) & (b != -1)
        i, j = a[m].ravel(), b[m].ravel()
        rows.append(np.concatenate([i, j])); cols.append(np.concatenate([j, i]))
    if rows:
        rows, cols = np.concatenate(rows), np.concatenate(cols)
        A = sp.coo_matrix((np.ones(rows.size, np.float32), (rows, cols)), shape=(idx.max()+1, idx.max()+1)).tocsr()
        d = np.asarray(A.sum(axis=1)).ravel()
        L_smooth = (sp.diags(d, 0) - A).astype(np.float32).tocsc()
    else:
        L_smooth = sp.csc_matrix((idx.max()+1, idx.max()+1), dtype=np.float32)

    selected_BOLD_flat = X.reshape(X.shape[0], -1).astype(np.float32)
    return L_task.astype(np.float32), L_var, L_smooth, selected_BOLD_flat

In [15]:
def objective_func(w, L_task, L_var, L_smooth, alpha_var, alpha_smooth):
    quad = (w.T @ np.diag(L_task) @ w + alpha_var * (w.T @ L_var @ w) + alpha_smooth * (w.T @ L_smooth @ w))
    return quad

In [16]:
def optimize_voxel_weights(L_task, L_var, L_smooth, alpha_var, alpha_smooth):
    L_total = np.diag(L_task) + alpha_var * L_var + alpha_smooth * L_smooth
    n = L_total.shape[0]
    L_total = np.nan_to_num(L_total)
    L_total = 0.5*(L_total + L_total.T) + 1e-8*np.eye(n)
    w = cp.Variable(n, nonneg=True)
    constraints = [cp.sum(w) == 1]

    # objective = cp.Minimize(cp.quad_form(w, L_total) + alpha_sparse * cp.norm1(w))
    objective = cp.Minimize(cp.quad_form(w, L_total))
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.OSQP, verbose=True)
    return w.value


In [17]:
def calculate_weight(param_grid, beta_valume_clean_2d, bold_data, anat_img, mask_2d, trial_len):
    kf = KFold(n_splits=2, shuffle=True, random_state=0)
    best_score = np.inf
    best_params = None
    num_trials = beta_valume_clean_2d.shape[-1]

    for a_var, a_smooth in product(*param_grid.values()):
        fold_scores = []
        print(f"a_var: {a_var}, a_smooth: {a_smooth}")
        count = 1

        for train_idx, val_idx in kf.split(np.arange(num_trials)):
            print(f"k-fold num: {count}")
            L_task_train, L_var_train, L_smooth_train, _ = calculate_matrices(beta_valume_clean_2d, bold_data, anat_img, mask_2d, train_idx, trial_len)
            w = optimize_voxel_weights(L_task_train, L_var_train, L_smooth_train, alpha_var=a_var, alpha_smooth=a_smooth)

            L_task_val, L_var_val, L_smooth_val, _ = calculate_matrices(beta_valume_clean_2d, bold_data, anat_img, mask_2d, val_idx, trial_len)

            fold_scores.append(objective_func(w, L_task_val, L_var_val, L_smooth_val, a_var, a_smooth))
            print(f"fold_scores: {fold_scores}")
            count += 1

        mean_score = np.mean(fold_scores)
        print(mean_score)
        if mean_score < best_score:
            best_score = mean_score
            best_params = (a_var, a_smooth)

    print("Best parameters:", best_params, "with CV loss:", best_score)
    return best_params, best_score

In [18]:
def select_opt_weight(selected_BOLD_data, weights, anat_img):
    y = selected_BOLD_data.T @ weights
    p95 = np.percentile(weights, 95)

    weight_volume = np.zeros(voxel_mask.shape, dtype=np.float32)
    weight_volume[voxel_mask] = weights.astype(np.float32)

    mask = np.zeros(voxel_mask.shape, dtype=bool)
    selected_weights = weights >= p95
    mask[voxel_mask] = selected_weights
    weight_volume[~mask] = 0.0

    masked_weights = np.where(mask, weight_volume, np.nan)
    weight_img = nib.Nifti1Image(masked_weights, affine=anat_img.affine)

    return weight_img, masked_weights, y


In [ ]:
param_grid = {
    "alpha_var":   [1.0],
    "alpha_smooth":[1.0]}

import scipy.sparse as sp
trial_len = 9
best_params, best_score = calculate_weight(param_grid, beta_valume_clean_2d, bold_data, anat_img, mask_2d, trial_len)

# L_task, L_var, L_smooth, selected_BOLD_data = calculate_matrices(beta_valume_clean_2d, bold_data, anat_img, mask_2d, None, trial_len)
# weights = optimize_voxel_weights(L_task, L_var, L_smooth, alpha_var=best_params[0], alpha_smooth=best_params[1])
# weight_img, masked_weights, y = select_opt_weight(selected_BOLD_data, weights, active_low_var_voxels.astype(bool), affine)
# print(y.shape)

a_var: 1.0, a_smooth: 1.0
k-fold num: 1


(CVXPY) Sep 26 06:42:50 AM: Your problem has 8563 variables, 1 constraints, and 0 parameters.


                                     CVXPY                                     
                                     v1.7.2                                    


(CVXPY) Sep 26 06:42:50 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 26 06:42:50 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 26 06:42:50 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Sep 26 06:42:50 AM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Sep 26 06:42:50 AM: Compiling problem (target solver=OSQP).
(CVXPY) Sep 26 06:42:50 AM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> OSQP
(CVXPY) Sep 26 06:42:50 AM: Applying reduction CvxAttr2Constr
(CVXPY) Sep 26 06:42:50 AM: Applying reduction Qp2SymbolicQp
(CVXPY) Sep 26 06:42:50 AM: Applying reduction QpMatrixStuffing
(CVXPY) Sep 26 06:42:50 AM: Applying reduction OSQP
(CVXPY) Sep 26 06:42:50 AM: Finished problem compilation (took 1.875e-01 seconds).
(CVXPY) Sep 26 06:42:50 AM: Invoking solver OSQP  to obtain a solutio

-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
-----------------------------------------------------------------
           OSQP v1.0.0  -  Operator Splitting QP Solver
              (c) The OSQP Developer Team
-----------------------------------------------------------------
problem:  variables n = 8563, constraints m = 8564
          nnz(P) + nnz(A) = 43297
settings: algebra = Built-in,
          OSQPInt = 4 bytes, OSQPFloat = 8 bytes,
          linear system solver = QDLDL v0.1.8,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-

...

(CVXPY) Sep 26 06:42:53 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 26 06:42:53 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 26 06:42:53 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Sep 26 06:42:53 AM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Sep 26 06:42:53 AM: Compiling problem (target solver=OSQP).
(CVXPY) Sep 26 06:42:53 AM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> OSQP
(CVXPY) Sep 26 06:42:53 AM: Applying reduction CvxAttr2Constr
(CVXPY) Sep 26 06:42:53 AM: Applying reduction Qp2SymbolicQp
(CVXPY) Sep 26 06:42:53 AM: Applying reduction QpMatrixStuffing
(CVXPY) Sep 26 06:42:53 AM: Applying reduction OSQP
(CVXPY) Sep 26 06:42:53 AM: Finished problem compilation (took 1.810e-01 seconds).
(CVXPY) Sep 26 06:42:53 AM: Invoking solver OSQP  to obtain a solutio

-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
-----------------------------------------------------------------
           OSQP v1.0.0  -  Operator Splitting QP Solver
              (c) The OSQP Developer Team
-----------------------------------------------------------------
problem:  variables n = 8563, constraints m = 8564
          nnz(P) + nnz(A) = 43297
settings: algebra = Built-in,
          OSQPInt = 4 bytes, OSQPFloat = 8 bytes,
          linear system solver = QDLDL v0.1.8,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-

(CVXPY) Sep 26 06:42:53 AM: Problem status: optimal
(CVXPY) Sep 26 06:42:53 AM: Optimal value: 6.719e-01
(CVXPY) Sep 26 06:42:53 AM: Compilation took 1.810e-01 seconds
(CVXPY) Sep 26 06:42:53 AM: Solver (including time spent in interface) took 2.453e-02 seconds


  50   6.7190e-01   1.58e-09   3.36e-08   3.57e-08   3.57e-08   1.00e-01    1.78e-02s
plsh   6.7190e-01   7.05e-15   2.44e-13   2.41e-13   2.44e-13   --------    2.29e-02s

status:               solved
solution polishing:   successful
number of iterations: 50
optimal objective:    0.6719
dual objective:       0.6719
duality gap:          2.4138e-13
primal-dual integral: 3.4109e+03
run time:             2.29e-02s
optimal rho estimate: 2.52e-02

-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
fold_scores: [np.float64(0.6830770153343357), np.float64(0.6615867410938843)]
0.67233187821411
Best parameters: (1.0, 1.0) with CV loss: 0.67233187821411


: 

In [ ]:
weights.shape

In [14]:
L_task, L_var, L_smooth, selected_BOLD_data = calculate_matrices(beta_valume_clean_2d, bold_data, anat_img, mask_2d, None, trial_len)

In [ ]:
weights = optimize_voxel_weights(L_task, L_var, L_smooth, alpha_var=best_params[0], alpha_smooth=best_params[1])

In [ ]:
print(weights.shape, np.max(weights), np.min(weights), np.sum(weights), np.median(weights), np.mean(weights))

In [ ]:
y = selected_BOLD_data.T @ weights
p95 = np.percentile(weights, 95)
selected_weights = weights >= p95

print(p95)

In [ ]:
np.sum(selected_weights)

In [ ]:
weight_img, masked_weights, y = select_opt_weight(selected_BOLD_data, weights, anat_img)

In [ ]:
plt.plot(weights)

In [1]:
from nilearn import datasets, image, masking

ho = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm', symmetric_split=True)

[fetch_atlas_harvard_oxford] Dataset found in /home/zkavian/nilearn_data/fsl


In [4]:
ho['labels']

['Background',
 'Left Frontal Pole',
 'Right Frontal Pole',
 'Left Insular Cortex',
 'Right Insular Cortex',
 'Left Superior Frontal Gyrus',
 'Right Superior Frontal Gyrus',
 'Left Middle Frontal Gyrus',
 'Right Middle Frontal Gyrus',
 'Left Inferior Frontal Gyrus, pars triangularis',
 'Right Inferior Frontal Gyrus, pars triangularis',
 'Left Inferior Frontal Gyrus, pars opercularis',
 'Right Inferior Frontal Gyrus, pars opercularis',
 'Left Precentral Gyrus',
 'Right Precentral Gyrus',
 'Left Temporal Pole',
 'Right Temporal Pole',
 'Left Superior Temporal Gyrus, anterior division',
 'Right Superior Temporal Gyrus, anterior division',
 'Left Superior Temporal Gyrus, posterior division',
 'Right Superior Temporal Gyrus, posterior division',
 'Left Middle Temporal Gyrus, anterior division',
 'Right Middle Temporal Gyrus, anterior division',
 'Left Middle Temporal Gyrus, posterior division',
 'Right Middle Temporal Gyrus, posterior division',
 'Left Middle Temporal Gyrus, temporooccipita